In [1]:
import pandas as pd
import datetime as dt

# ---- Step 1: Load the data ----
df = pd.read_csv(
    "combined_unique.csv",
    usecols=["Invoice", "Customer ID", "InvoiceDate", "Price", "Quantity", "Description"],
    encoding="latin1"
)

# ---- Step 2: Clean data ----
df = df.dropna(subset=["Customer ID", "InvoiceDate", "Description"])
df["Customer ID"] = df["Customer ID"].astype(int)
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

# Remove negative quantities (returns)
df = df[df["Quantity"] > 0]

# ---- Step 3: Create Total Amount column ----
df["TotalAmount"] = df["Quantity"] * df["Price"]

# ---- Step 4: Define a snapshot date ----
snapshot_date = df["InvoiceDate"].max() + dt.timedelta(days=1)

# ---- Step 5: Compute RFM metrics ----
rfm = df.groupby("Customer ID").agg({
    "InvoiceDate": lambda x: (snapshot_date - x.max()).days,  # Recency
    "Invoice": "nunique",  # Frequency
    "TotalAmount": "sum"   # Monetary
}).reset_index()

rfm.rename(columns={
    "InvoiceDate": "Recency",
    "Invoice": "Frequency",
    "TotalAmount": "Monetary"
}, inplace=True)

# ---- Step 6: Rank customers (1–5 scale, higher = better) ----
rfm["R_rank"] = pd.qcut(rfm["Recency"], 5, labels=[5,4,3,2,1])
rfm["F_rank"] = pd.qcut(rfm["Frequency"].rank(method="first"), 5, labels=[1,2,3,4,5])
rfm["M_rank"] = pd.qcut(rfm["Monetary"], 5, labels=[1,2,3,4,5])

# Combine to get RFM score
rfm["RFM_Score"] = rfm[["R_rank","F_rank","M_rank"]].sum(axis=1).astype(int)

print("Top 10 RFM customers:")
display(rfm.sort_values("RFM_Score", ascending=False).head(10))


/tmp/ipython-input-3613270418.py:5: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


Top 10 RFM customers:


,Customer ID,Recency,Frequency,Monetary,R_rank,F_rank,M_rank,RFM_Score
2658,15031,4,14,3102.94,5,5,5,15
2661,15034,16,14,8231.20,5,5,5,15
5838,18245,7,20,6324.98,5,5,5,15
5834,18241,10,21,2986.99,5,5,5,15
2666,15039,10,93,39141.48,5,5,5,15
5865,18272,3,9,4385.98,5,5,5,15
2650,15023,3,20,11578.65,5,5,5,15
2651,15024,10,16,3559.39,5,5,5,15
1781,14146,9,12,9450.30,5,5,5,15
1791,14156,10,156,313437.62,5,5,5,15


In [2]:
# ---- Step 7: Identify top customers ----
top_customers = rfm[rfm["RFM_Score"] >= 13]["Customer ID"]

# ---- Step 8: Find products they recently bought ----
recent_top_purchases = df[df["Customer ID"].isin(top_customers)]

# ---- Step 9: Rank products by total sales to top customers ----
product_reco = (
    recent_top_purchases.groupby("Description")
    .agg({"Quantity": "sum", "TotalAmount": "sum"})
    .sort_values("Quantity", ascending=False)
    .head(10)
    .reset_index()
)

print("Top Recommended Products for Active Customers:")
display(product_reco)


Top Recommended Products for Active Customers:


,Description,Quantity,TotalAmount
0,WHITE HANGING HEART T-LIGHT HOLDER,66586,178987.06
1,WORLD WAR 2 GLIDERS ASSTD DESIGNS,66203,15714.77
2,JUMBO BAG RED RETROSPOT,63968,115284.28
3,ASSORTED COLOUR BIRD ORNAMENT,63250,99619.54
4,BROCADE RING PURSE,54862,11354.00
5,PACK OF 60 PINK PAISLEY CAKE CASES,45636,21637.55
6,60 TEATIME FAIRY CAKE CASES,39536,18952.68
7,PACK OF 72 RETROSPOT CAKE CASES,36528,17264.05
8,PACK OF 72 RETRO SPOT CAKE CASES,36439,17245.93
9,SMALL POPCORN HOLDER,34791,26190.53
